In [3]:
import pandas as pd 
import numpy as np
import function
import matplotlib.pyplot as plt

In [4]:
train_df = pd.read_csv("./train.csv")
print(train_df.columns)
print(train_df.shape)

Index(['image_id', 'class_name', 'class_id', 'rad_id', 'x_min', 'y_min',
       'x_max', 'y_max'],
      dtype='object')
(67914, 8)


In [5]:
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [6]:
image_id = train_df["image_id"]
class_name =  train_df["class_name"]
class_id = train_df["class_id"] 
rad_id = train_df["rad_id"]
x_min = train_df["x_min"]
y_min = train_df["y_min"]
x_max = train_df["x_max"]
y_max = train_df["y_max"]

train_df = train_df.fillna(0)
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,0.0,0.0,0.0,0.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,0.0,0.0,0.0,0.0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,0.0,0.0,0.0,0.0


In [7]:
img_len = len(train_df["image_id"].value_counts())
print(img_len)

img_name_list = train_df["image_id"].value_counts().index.tolist()
print(len(img_name_list))
print(len(set(img_name_list)))

15000
15000
15000


In [10]:
limit = 0.4
idx_list = [] # iou 이상인 index끼리 담아 놓은

for img_name in img_name_list:
    indexes = train_df[train_df["image_id"] == img_name].index
    for i in range(len(indexes)-1):
        for j in range(i+1,len(indexes)):
            rect1 = (x_min[indexes[i]], y_min[indexes[i]], x_max[indexes[i]], y_max[indexes[i]])
            rect2 = (x_min[indexes[j]], y_min[indexes[j]], x_max[indexes[j]], y_max[indexes[j]])
            iou = function.get_iou(rect1, rect2)
            if(iou >= limit):
                idx_list.append((indexes[i], indexes[j])) 
                

In [11]:
print(len(idx_list))
idx_list[0:10]

29677


[(757, 37684),
 (1906, 3318),
 (1906, 41550),
 (2087, 50893),
 (2489, 49434),
 (3318, 41550),
 (12042, 66294),
 (12234, 13136),
 (14463, 59265),
 (16073, 63260)]

In [29]:
overlap_df = pd.DataFrame(index=img_name_list, columns=['overlap_class_name', 'overlap_class_id', 'overlap_rad_id', 'overlap_index'])
print(overlap_df.shape)
overlap_df.replace(np.NAN, "", inplace=True)
overlap_df

(15000, 4)


,overlap_class_name,overlap_class_id,overlap_rad_id,overlap_index
03e6ecfa6f6fb33dfeac6ca4f9b459c9,,,,
fa109c087e46fe1ea27e48ce6d154d2f,,,,
e31be972e181987a8600a8700c1ebe88,,,,
3a302fbbbf3364aa1a7731b59e6b98ec,,,,
ecf474d5d4f65d7a3e23370a68b8c6a0,,,,
...,...,...,...,...
10ab779d6ca517c05e324e6386218974,,,,
e27b0c92b056198188b7f73b95f555a9,,,,
62f4825401cef60e0a9c3a3d36dd30ee,,,,
d0bccf34602159efb1fb3c3425ff59f2,,,,


In [31]:
for idx in idx_list: # ( a, b )
    df_idx1 = idx[0]
    df_idx2 = idx[1]
    ocn = overlap_df.loc[[image_id[df_idx1]]]["overlap_class_name"][0] +  str(train_df.loc[df_idx1][1]) + '-' + str(train_df.loc[df_idx2][1]) + '-'
    oci = overlap_df.loc[[image_id[df_idx1]]]["overlap_class_id"][0] +  str(train_df.loc[df_idx1][2]) + '-' + str(train_df.loc[df_idx2][2]) + '-'
    ori = overlap_df.loc[[image_id[df_idx1]]]["overlap_rad_id"][0] +  str(train_df.loc[df_idx1][3]) + '-' + str(train_df.loc[df_idx2][3]) + '-'
    oi = overlap_df.loc[[image_id[df_idx1]]]["overlap_index"][0] +  str(df_idx1) + '-' + str(df_idx2) + '-'
    overlap_df.loc[image_id[df_idx1]] = [ocn, oci, ori, oi]
overlap_df

,overlap_class_name,overlap_class_id,overlap_rad_id,overlap_index
03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-5-8-8-8-8-8-8-8-8-8-8-8-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R10-R9-R8-R9-R8-R9-R8-R9-R...,757-37684-1906-3318-1906-41550-2087-50893-2489...
fa109c087e46fe1ea27e48ce6d154d2f,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-8-8-8-8-8-8-8-7-8-7-8-7-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R9-R8-R10-R8-R10-R9-R8-R9-...,7301-51569-7627-10763-7627-45183-9588-65003-10...
e31be972e181987a8600a8700c1ebe88,Nodule/Mass-Nodule/Mass-Nodule/Mass-Calcificat...,8-8-8-2-11-2-11-2-11-11-11-2-11-11-11-2-2-11-2...,R10-R9-R10-R8-R8-R8-R10-R10-R10-R8-R10-R9-R10-...,496-13033-496-52024-606-65305-1156-17557-1156-...
3a302fbbbf3364aa1a7731b59e6b98ec,Nodule/Mass-Nodule/Mass-Nodule/Mass-Lung Opaci...,8-8-8-7-8-7-8-8-8-7-8-8-8-7-8-7-8-8-8-7-8-8-10...,R9-R8-R9-R10-R9-R9-R9-R10-R9-R10-R9-R8-R9-R9-R...,237-20856-237-38643-237-56033-237-59273-1432-2...
ecf474d5d4f65d7a3e23370a68b8c6a0,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-3-3-8-8-8-8-8-8-11-11-8-8-8-8-0-0-...,R8-R9-R9-R8-R9-R8-R9-R10-R9-R8-R9-R10-R9-R8-R9...,1816-51356-5321-7248-7599-55069-9079-40261-100...
...,...,...,...,...
10ab779d6ca517c05e324e6386218974,,,,
e27b0c92b056198188b7f73b95f555a9,,,,
62f4825401cef60e0a9c3a3d36dd30ee,,,,
d0bccf34602159efb1fb3c3425ff59f2,,,,


In [32]:
overlap_df.replace("", np.NAN, inplace=True)

In [33]:
overlap_df

,overlap_class_name,overlap_class_id,overlap_rad_id,overlap_index
03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-5-8-8-8-8-8-8-8-8-8-8-8-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R10-R9-R8-R9-R8-R9-R8-R9-R...,757-37684-1906-3318-1906-41550-2087-50893-2489...
fa109c087e46fe1ea27e48ce6d154d2f,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-8-8-8-8-8-8-8-8-7-8-7-8-7-8-8-8-8-...,R9-R8-R10-R8-R10-R9-R9-R8-R10-R8-R10-R9-R8-R9-...,7301-51569-7627-10763-7627-45183-9588-65003-10...
e31be972e181987a8600a8700c1ebe88,Nodule/Mass-Nodule/Mass-Nodule/Mass-Calcificat...,8-8-8-2-11-2-11-2-11-11-11-2-11-11-11-2-2-11-2...,R10-R9-R10-R8-R8-R8-R10-R10-R10-R8-R10-R9-R10-...,496-13033-496-52024-606-65305-1156-17557-1156-...
3a302fbbbf3364aa1a7731b59e6b98ec,Nodule/Mass-Nodule/Mass-Nodule/Mass-Lung Opaci...,8-8-8-7-8-7-8-8-8-7-8-8-8-7-8-7-8-8-8-7-8-8-10...,R9-R8-R9-R10-R9-R9-R9-R10-R9-R10-R9-R8-R9-R9-R...,237-20856-237-38643-237-56033-237-59273-1432-2...
ecf474d5d4f65d7a3e23370a68b8c6a0,Nodule/Mass-Nodule/Mass-Nodule/Mass-Nodule/Mas...,8-8-8-8-8-8-3-3-8-8-8-8-8-8-11-11-8-8-8-8-0-0-...,R8-R9-R9-R8-R9-R8-R9-R10-R9-R8-R9-R10-R9-R8-R9...,1816-51356-5321-7248-7599-55069-9079-40261-100...
...,...,...,...,...
10ab779d6ca517c05e324e6386218974,NaN,NaN,NaN,NaN
e27b0c92b056198188b7f73b95f555a9,NaN,NaN,NaN,NaN
62f4825401cef60e0a9c3a3d36dd30ee,NaN,NaN,NaN,NaN
d0bccf34602159efb1fb3c3425ff59f2,NaN,NaN,NaN,NaN


In [34]:
print(overlap_df.isnull())
overlap_df.isnull().sum()
# 15000 - 10648 = 4352개 이미지에 겹치는 부분 있음

                                  overlap_class_name  overlap_class_id  \
03e6ecfa6f6fb33dfeac6ca4f9b459c9               False             False   
fa109c087e46fe1ea27e48ce6d154d2f               False             False   
e31be972e181987a8600a8700c1ebe88               False             False   
3a302fbbbf3364aa1a7731b59e6b98ec               False             False   
ecf474d5d4f65d7a3e23370a68b8c6a0               False             False   
...                                              ...               ...   
10ab779d6ca517c05e324e6386218974                True              True   
e27b0c92b056198188b7f73b95f555a9                True              True   
62f4825401cef60e0a9c3a3d36dd30ee                True              True   
d0bccf34602159efb1fb3c3425ff59f2                True              True   
0f841fcf5b68b1fbda85e6dcdcb3c4fd                True              True   

                                  overlap_rad_id  overlap_index  
03e6ecfa6f6fb33dfeac6ca4f9b459c9           Fa

overlap_class_name    10648
overlap_class_id      10648
overlap_rad_id        10648
overlap_index         10648
dtype: int64

In [35]:
# 사진
import cv2
img = function.read_xray('./dataset/03e6ecfa6f6fb33dfeac6ca4f9b459c9.dicom')
img = cv2.merge((img, img, img))

tlist = overlap_df.loc["03e6ecfa6f6fb33dfeac6ca4f9b459c9"]["overlap_index"].split('-')
tlist = tlist[0:-1]
tlist = [int(i) for i in tlist]
tlist = set(tlist)

all_list = image_id[image_id == "03e6ecfa6f6fb33dfeac6ca4f9b459c9"].index
all_list = set(all_list)
diff_list = all_list.difference(tlist)


for index in tlist:
    cv2.rectangle(img, (int(x_min[index]), int(y_min[index])), (int(x_max[index]), int(y_max[index])), function.red, 3) 
    cv2.putText(img, str(class_name[index]), (int(x_min[index]), int(y_min[index]-3)),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255) , 2)

for index in diff_list:
    cv2.rectangle(img, (int(x_min[index]), int(y_min[index])), (int(x_max[index]), int(y_max[index])), function.blue, 3) 
    cv2.putText(img, str(class_name[index]), (int(x_min[index]), int(y_min[index]-3)),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255) , 2)

img = function.FitToWindowSize(img)
cv2.imshow("", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
overlap_df = overlap_df.dropna()
overlap_df.shape

(4352, 4)

In [37]:
overlap_index_list = overlap_df.index

In [53]:
chk_list = []
for index in overlap_index_list:
    index_group = overlap_df.loc[index].overlap_index
    index_set = index_group.split('-')[0:-1]
    index_set = list(set(index_set))
    for i in range(len(index_set)-1):
        x = int(index_set[i])
        txt = class_name[x]
        print('---------------------------')
        print(f'x, {x} {txt}')
        for j in range(i+1, len(index_set)):
            y = int(index_set[j])
            print(f'y, {y} {class_name[y]}')
            if class_name[y] not in txt:  
                txt = txt + ' + ' + class_name[y]
        print(f'final {txt}')
        if txt != class_name[x]:
            chk_list.append(txt)

lmonary fibrosis
y, 67436 Pulmonary fibrosis
final Pulmonary fibrosis
---------------------------
x, 20999 Cardiomegaly
y, 8032 Cardiomegaly
y, 41391 Cardiomegaly
final Cardiomegaly
---------------------------
x, 8032 Cardiomegaly
y, 41391 Cardiomegaly
final Cardiomegaly
---------------------------
x, 29089 Aortic enlargement
y, 834 Aortic enlargement
y, 19865 Aortic enlargement
final Aortic enlargement
---------------------------
x, 834 Aortic enlargement
y, 19865 Aortic enlargement
final Aortic enlargement
---------------------------
x, 44544 Cardiomegaly
y, 5820 Cardiomegaly
y, 1450 Cardiomegaly
final Cardiomegaly
---------------------------
x, 5820 Cardiomegaly
y, 1450 Cardiomegaly
final Cardiomegaly
---------------------------
x, 63507 Aortic enlargement
y, 45310 Aortic enlargement
y, 40200 Aortic enlargement
final Aortic enlargement
---------------------------
x, 45310 Aortic enlargement
y, 40200 Aortic enlargement
final Aortic enlargement
---------------------------
x, 24724 Aor

In [54]:
chk_list[0:10]

['Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'ILD + Nodule/Mass',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD',
 'Nodule/Mass + ILD']

In [55]:
for i in range(len(chk_list)):
    tmp = sorted(chk_list[i].split(' + '))
    result = tmp[0]
    for j in range(1,len(tmp)):
        result = result + ' + ' + tmp[j] 
    chk_list[i] = result
chk_list[0:10]

['ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass',
 'ILD + Nodule/Mass']

In [76]:
from collections import Counter

result = Counter(chk_list)
print(result)

keys = []
results = []
for key in result:
    print(key, result[key])
    keys.append(key)
    results.append(result[key])

rtic enlargement + Cardiomegaly + ILD 23
Aortic enlargement + ILD 42
ILD + Infiltration + Pleural thickening + Pulmonary fibrosis 3
Aortic enlargement + ILD + Infiltration + Pleural effusion + Pleural thickening 1
Aortic enlargement + ILD + Infiltration + Pleural thickening 7
Aortic enlargement + ILD + Infiltration 21
Cardiomegaly + ILD + Infiltration + Pleural effusion + Pleural thickening 3
Cardiomegaly + ILD + Pleural effusion + Pleural thickening 8
Cardiomegaly + ILD 15
Aortic enlargement + Infiltration + Lung Opacity + Other lesion + Pleural effusion + Pulmonary fibrosis 2
Aortic enlargement + Infiltration + Lung Opacity + Other lesion + Pulmonary fibrosis 3
Aortic enlargement + Infiltration + Other lesion + Pulmonary fibrosis 7
Aortic enlargement + Infiltration + Lung Opacity + Pleural thickening + Pulmonary fibrosis 2
Aortic enlargement + ILD + Infiltration + Lung Opacity + Pleural effusion + Pleural thickening + Pulmonary fibrosis 5
Aortic enlargement + ILD + Lung Opacity + Ple

In [83]:
result_df = pd.DataFrame(list(zip(keys, results)), columns =['overlap', 'count'])
result_df

,overlap,count
0,ILD + Nodule/Mass,36
1,Consolidation + Lung Opacity + Nodule/Mass,42
2,Consolidation + Nodule/Mass,26
3,Aortic enlargement + Calcification + Lung Opac...,27
4,Aortic enlargement + Calcification + Lung Opac...,13
...,...,...
1247,Atelectasis + ILD,1
1248,Cardiomegaly + Pleural thickening + Pneumothorax,1
1249,Cardiomegaly + Pneumothorax,2
1250,Calcification + Infiltration + Lung Opacity + ...,1


In [84]:
result_df.sort_values(by=['count'], axis=0, ascending=False)

,overlap,count
221,Aortic enlargement + Cardiomegaly,3847
94,Pleural effusion + Pleural thickening,479
149,Pleural thickening + Pulmonary fibrosis,310
159,Aortic enlargement + Pleural thickening,300
207,Aortic enlargement + Pulmonary fibrosis,277
...,...,...
952,Calcification + Consolidation + ILD + Pleural ...,1
950,Calcification + Lung Opacity + Nodule/Mass + O...,1
348,Atelectasis + Calcification + Lung Opacity + N...,1
945,Consolidation + Infiltration + Lung Opacity + ...,1
